In [1]:
import pandas as pd
import numpy as np
import requests
import time
from bs4 import BeautifulSoup
pd.set_option('display.max_columns', None)

In [2]:
#SCRAPE ALL PLAYER STATS

#add your base url with curly braces where year will be inserted
base_URL = 'https://www.basketball-reference.com/leagues/NBA_{}_per_game.html'

#identify start and end years
start_year = 1956
end_year = 2023

#create empty list
dfs = []

request_delay = 10

#Loop through every year
for year in range(start_year, end_year + 1):

    url = base_URL.format(year)

    res = requests.get(url)
    
    #checking if successful request
    if res.status_code == 200:

        #uses beautiful soup to pull in content and find table
        soup = BeautifulSoup(res.text, 'html.parser')

        table = soup.find('table', {'class': 'sortable'})

        #checking if a table is found
        if table:

            #create a blank list of table data
            table_data = []
            #find all <tr> elements in the table and iterate through them
            for row in table.find_all('tr'):
                #for each row find all <th> and <td> elements
                row_data = [cell.get_text(strip=True) for cell in row.find_all(['th' and 'td'])]
                #append row data into our table_data list
                table_data.append(row_data)

            #check if there is table data
            if table_data:

                #checking for headers
                if table_data[0]:
                    df = pd.DataFrame(table_data[1:], columns=table_data[0])
                else:
                    df = pd.DataFrame(table_data)
                #adding year column using our year var
                df['Year'] = year
                #appends this df onto our larger df
                dfs.append(df)
                #basketball-reference rate limits ips that make more than 20 bot requests in a minute so this delay avoids that
                time.sleep(request_delay) 
                
# Concatenate all DataFrames into a single DataFrame
combined_df = pd.concat(dfs, ignore_index=True)

# Export the combined DataFrame to a CSV file
#combined_df.to_csv('basketball_reference_per_game_stats.csv', index=False)

In [3]:
#remove null rows
df_final = combined_df.dropna(subset=[0])

In [4]:
#add headers
headers = ['PLAYER', 'POS', 'AGE', 'TEAM', 'GP', 'GS', 'MPG', 'FGM', 'FGA', 'FG%', '3PM', '3PA', '3P%', '2PM', '2PA', '2P%', 'eFG%', 'FTM', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TO', 'PF', 'PTS', 'Year']

df_final.columns = headers

#add column that will be used to merge scraped data later
df_final['Merge Field'] = df_final['PLAYER'] + df_final['POS'] + df_final['Year'].astype(str)

#send to CSV (uncomment to send only this file to csv)
#df_final.to_csv('Documents/historical_nba_data.csv', index=False)

/var/folders/w2/65dy7vrs7hl_vr43jvzjdk340000gn/T/ipykernel_74917/3019696737.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['Merge Field'] = df_final['PLAYER'] + df_final['POS'] + df_final['Year'].astype(str)


,PLAYER,POS,AGE,TEAM,GP,GS,MPG,FGM,FGA,FG%,3PM,3PA,3P%,2PM,2PA,2P%,eFG%,FTM,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TO,PF,PTS,Year,Merge Field
1,Paul Arizin*,SF,27,PHW,72,,37.8,8.6,19.1,.448,,,,8.6,19.1,.448,.448,7.0,8.7,.810,,,7.5,2.6,,,,3.9,24.2,1956,Paul Arizin*SF1956
2,Jesse Arnelle,PF,22,FTW,31,,13.2,1.7,5.3,.317,,,,1.7,5.3,.317,.317,1.4,2.2,.623,,,5.5,0.6,,,,1.9,4.7,1956,Jesse ArnellePF1956
3,Dick Atha,SG,24,NYK,25,,11.5,1.4,3.5,.409,,,,1.4,3.5,.409,.409,0.8,1.1,.778,,,1.7,1.3,,,,1.6,3.7,1956,Dick AthaSG1956
4,Jim Baechtold,SF,28,NYK,70,,24.8,3.8,9.9,.386,,,,3.8,9.9,.386,.386,3.3,4.2,.801,,,3.1,2.3,,,,2.2,11.0,1956,Jim BaechtoldSF1956
5,Ernie Barrett,SG,26,BOS,72,,20.2,2.9,7.4,.388,,,,2.9,7.4,.388,.388,1.3,1.6,.788,,,3.4,2.4,,,,2.6,7.0,1956,Ernie BarrettSG1956
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29015,Thaddeus Young,PF,34,TOR,54,9,14.7,2.0,3.7,.545,0.1,0.6,.176,1.9,3.0,.622,.561,0.3,0.5,.692,1.3,1.8,3.1,1.4,1.0,0.1,0.8,1.6,4.4,2023,Thaddeus YoungPF2023
29016,Trae Young,PG,24,ATL,73,73,34.8,8.2,19.0,.429,2.1,6.3,.335,6.1,12.7,.476,.485,7.8,8.8,.886,0.8,2.2,3.0,10.2,1.1,0.1,4.1,1.4,26.2,2023,Trae YoungPG2023
29017,Omer Yurtseven,C,24,MIA,9,0,9.2,1.8,3.0,.593,0.3,0.8,.429,1.4,2.2,.650,.648,0.6,0.7,.833,0.9,1.7,2.6,0.2,0.2,0.2,0.4,1.8,4.4,2023,Omer YurtsevenC2023
29018,Cody Zeller,C,30,MIA,15,2,14.5,2.5,3.9,.627,0.0,0.1,.000,2.5,3.8,.649,.627,1.6,2.3,.686,1.7,2.6,4.3,0.7,0.2,0.3,0.9,2.2,6.5,2023,Cody ZellerC2023


In [5]:
#SCRAPE MVPs

#url of table being scraped
url = "https://www.basketball-reference.com/leagues/"

#get request on url to pull data
res = requests.get(url)

#checks if get request is successful
if res.status_code == 200:

    #uses beautiful soup to pull in content and find table
    soup = BeautifulSoup(res.content, 'html.parser')

    table = soup.find('table', {'class': 'sortable'})

    #checks if a table is found
    if table:
        #creates empty list to hold our data
        table_data = []
        #initiates variable to hold column num int
        max_cols = 0
        #loops through the table to find all table rows
        for row in table.find_all('tr'):
            #for each row finds all table headers and data
            row_data = [cell.get_text(strip=True) for cell in row.find_all(['th', 'td'])]
            #appends all data to our table_data list
            table_data.append(row_data)
            #sets max columns to the maximum number of columns for any single row
            max_cols = max(max_cols, len(row_data))

        #checks if table data does not have headers
        if not table_data[0]:
            #pulls from row 2 on
            table_data = table_data[1:]

        #checks if there is table data
        if table_data:
            #adds a blank into any rows that don't match the max number of columns
            for i, row in enumerate(table_data):
                table_data[i] = row + [''] * (max_cols - len(row))
        
        
    #convert our table into a data frame
    df1 = pd.DataFrame(table_data[1:], columns=table_data[0])
        
    time.sleep(request_delay) 
    #print the table (not really needed)
    #print(df1)

    #send the data to a csv
    #df.to_csv('Documents/2023.csv')

In [6]:
#add headers to df
headers = ['Season', 'Lg', 'Champion', 'MVP', 'ROY', 'Scoring Leader', 'Rebound Leader', 'Assist Leader', 'Win Shares']

df1.columns = headers

#drop first row as it is blank
df1 = df1.drop(0)

#creating merge field for later
df1['year_start'] = df1['Season'].str.slice(0, 4)
df1['Merge Field'] = df1['year_start'].astype(str) + df1['MVP']

,Season,Lg,Champion,MVP,ROY,Scoring Leader,Rebound Leader,Assist Leader,Win Shares,year_start,Merge Field
1,2023-24,NBA,,,V. Wembanyama,L. Dončić(2370),D. Sabonis(1120),T. Haliburton(752),N. Jokić(17.0),2023,2023
2,2022-23,NBA,Denver Nuggets,J. Embiid,P. Banchero,J. Tatum(2225),D. Sabonis(973),T. Young(741),N. Jokić(14.9),2022,2022J. Embiid
3,2021-22,NBA,Golden State Warriors,N. Jokić,S. Barnes,T. Young(2155),N. Jokić(1019),T. Young(737),N. Jokić(15.2),2021,2021N. Jokić
4,2020-21,NBA,Milwaukee Bucks,N. Jokić,L. Ball,S. Curry(2015),R. Gobert(960),R. Westbrook(763),N. Jokić(15.6),2020,2020N. Jokić
5,2019-20,NBA,Los Angeles Lakers,G. Antetokounmpo,J. Morant,J. Harden(2335),R. Gobert(916),L. James(684),J. Harden(13.1),2019,2019G. Antetokounmpo
...,...,...,...,...,...,...,...,...,...,...,...
83,1950-51,NBA,Rochester Royals,,P. Arizin,G. Mikan(1932),D. Schayes(1080),A. Phillip(414),G. Mikan(23.4),1950,1950
84,1949-50,NBA,Minneapolis Lakers,,A. Groza,G. Mikan(1865),,D. McGuire(386),G. Mikan(21.1),1949,1949
85,1948-49,BAA,Minneapolis Lakers,,H. Shannon,G. Mikan(1698),,B. Davies(321),G. Mikan(20.9),1948,1948
86,1947-48,BAA,Baltimore Bullets,,P. Hoffman,M. Zaslofsky(1007),,H. Dallmar(120),B. Feerick(10.5),1947,1947


In [7]:
#SCRAPE SHOOTING STATS

#add your base url with curly braces where year will be inserted
base_URL = 'https://www.basketball-reference.com/leagues/NBA_{}_shooting.html'

#identify start and end years
start_year = 1998
end_year = 2023

#create empty list
dfs = []

request_delay = 10

#Loop through every year
for year in range(start_year, end_year + 1):

    url = base_URL.format(year)

    res = requests.get(url)
    
    #checking if successful request
    if res.status_code == 200:

        #uses beautiful soup to pull in content and find table
        soup = BeautifulSoup(res.text, 'html.parser')

        table = soup.find('table', {'class': 'sortable'})
        
        #checks if a table is found
        if table:
            #creates empty list to hold our data
            table_data = []
            #find all <tr> elements in the table and iterate through them
            for row in table.find_all('tr'):
                #for each row find all <th> and <td> elements
                row_data = [cell.get_text(strip=True) for cell in row.find_all(['th' and 'td'])]
                #append row data into our table_data list
                table_data.append(row_data)
                
            #check if there is table data
            if table_data:

                #checking for headers
                if table_data[0]:
                    df = pd.DataFrame(table_data[1:], columns=table_data[0])
                else:
                    df = pd.DataFrame(table_data)

                #adding year column
                df['Year'] = year

                #appending df to larger merged df created outside of loop
                dfs.append(df)

                #basketball-reference rate limits ips that make more than 20 bot requests in a minute so this delay avoids that
                time.sleep(request_delay) 
                
# Concatenate all DataFrames into a single DataFrame
combined_df2 = pd.concat(dfs, ignore_index=True)

# Export the combined DataFrame to a CSV file (if you want just this data)
#combined_df.to_csv('basketballref_shooting_stats.csv', index=False)

In [8]:
#add headers to df
headers = ['Player','POS', 'AGE', 'TEAM', 'GP', 'MP', 'FG%', 'Dist','blank1', '% of Shots from 2PT', 
           '% of Shots from 0-3ft','% of Shots from 3-10ft','% of Shots from 10-16ft',
           '% of Shots from 16+ft', '% of Shots from 3PT', 'blank2',
           'FG% 2PT','FG% 0-3ft','FG% 3-10ft','FG% 10-16ft','FG% 16+ft','FG% 3PT', 'blank3',
           '% assisted 2PT Shots', '% assisted 3PT Shots', 'blank4','FG% Dunks', 'Count Dunks', 'blank5',
          '% of 3PT attempts from corner', 'Corner 3PT %', 'blank6', 'Heaves Attempted', 'Heaves Made','Year']

combined_df2.columns = headers

#remove empty rows on top
rows_to_remove = [0,1]

combined_df2 = combined_df2.drop(rows_to_remove)

#remove blank columns
columns_to_remove = ['blank1','blank2','blank3','blank4','blank5','blank6']

combined_df2 = combined_df2.drop(columns=columns_to_remove, axis=1)

#creating combined merge field for later merge
combined_df2['Merge Field'] = combined_df2['Player'] + combined_df2['POS'] + combined_df2['Year'].astype(str)

In [9]:
merged_df = pd.merge(df_final, combined_df2, on='Merge Field', how='left')

In [10]:
#remove and rename columns
columns_to_remove = ['POS_y','AGE_y','TEAM_y','GP_y','FG%_y','Year_y','Merge Field']

merged_df = merged_df.drop(columns=columns_to_remove, axis=1)

merged_df = merged_df.rename(columns={'POS_x': 'POS', 'AGE_x':'AGE', 'TEAM_x':'TEAM', 'GP_x': 'GP', 'FG%_x': 'FG%', 'Year_x': 'Year'})

#pull first initial of name
merged_df['FirstInit'] = merged_df['PLAYER'].str.slice(0,1) + '.'

#split off last name
merged_df['LastName'] = merged_df['PLAYER'].str.split().str[-1]

merged_df['Merge Field'] = merged_df['Year'].astype(str) + merged_df['FirstInit'] + ' ' + merged_df['LastName']

In [12]:
#creating final merged df with all 3 df
NBA_df = pd.merge(merged_df, df1, on='Merge Field', how='left')

#removing unnecessary columns
final_columns_to_remove = ['Merge Field','MergeField','Lg','year_start']

#NBA_df = NBA_df.drop(columns=final_columns_to_remove, axis=1)

#shows df
NBA_df

KeyError: "['MergeField'] not found in axis"

In [ ]:
#exporting to csv in documents folder
NBA_df.to_csv('Documents/HistoricalNBAData.csv', index=False)